In [2]:
import pandas as pd
#Z:\Asset Management\FUNDOS e CLUBES\Gerencial\dashboard LFT.xlsx
dados = pd.read_excel(r'Z:\Asset Management\FUNDOS e CLUBES\Gerencial\dashboard LFT.xlsx', sheet_name='Historico preços')
#Renomear a coluna 'Unnamed: 0' para 'Data'
dados.rename(columns={'Unnamed: 0': 'Data'}, inplace=True)
#Dropar as primeiras duas linhas
dados.drop(index=[0, 1], inplace=True)
#Ficar somente com a coluna 'BLFT 0 06/01/30'
dados = dados[['Data', 'BLFT 0 06/01/30']]
#Renomear a coluna 'BLFT 0 06/01/30' para 'RetornoLFT'
dados.rename(columns={'BLFT 0 06/01/30': 'RetornoLFT'}, inplace=True)

dados.to_csv('Dados/dados_lft.csv', index=False)
dados

,Data,RetornoLFT
2,2024-07-14 00:00:00,NaN
3,2024-07-15 00:00:00,14927.313594
4,2024-07-16 00:00:00,14933.628116
5,2024-07-17 00:00:00,14940.367855
6,2024-07-18 00:00:00,14947.563505
...,...,...
363,2025-07-10 00:00:00,16812.95587
364,2025-07-11 00:00:00,16821.903561
365,2025-07-12 00:00:00,NaN
366,2025-07-13 00:00:00,NaN


In [ ]:
#Renomear a coluna 'Unnamed: 0' para 'Data'
dados.rename(columns={'Unnamed: 0': 'Data'}, inplace=True)
#Dropar as primeiras duas linhas
dados.drop(index=[0, 1], inplace=True)
#Ficar somente com a coluna 'BLFT 0 06/01/30'
dados = dados[['Data', 'BLFT 0 06/01/30']]
#Renomear a coluna 'BLFT 0 06/01/30' para 'RetornoLFT'
dados.rename(columns={'BLFT 0 06/01/30': 'RetornoLFT'}, inplace=True)

dados.to_csv('dados_lft.csv', index=False)

C:\Users\emanuel.cabral\AppData\Local\Temp\ipykernel_30364\1061845245.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dados.rename(columns={'BLFT 0 06/01/30': 'RetornoLFT'}, inplace=True)


,Data,RetornoLFT
2,2024-07-10 00:00:00,16775.731744
3,2024-07-11 00:00:00,16776.137799
4,2024-07-12 00:00:00,16776.628449
5,2024-07-13 00:00:00,NaN
6,2024-07-14 00:00:00,NaN
...,...,...
363,2025-07-06 00:00:00,NaN
364,2025-07-07 00:00:00,16822.546481
365,2025-07-08 00:00:00,16822.800265
366,2025-07-09 00:00:00,16822.952535


In [4]:
import requests
SGS_ID = 12
url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs/{SGS_ID}/dados?formato=json"

resp = requests.get(
    url,
    headers={"Accept": "application/json", "User-Agent": "Mozilla/5.0"},
    timeout=10,
)
resp.raise_for_status()

df = (pd.DataFrame(resp.json())              # [{'data':'01/07/2024', 'valor':'0.0155'}, …]
        .rename(columns={"data": "Data", "valor": "cdi"})
        .assign(Data=lambda d: pd.to_datetime(d["Data"], dayfirst=True),
                cdi =lambda d: pd.to_numeric(d["cdi"],
                                                errors="coerce") / 100)  # → decimal
        .dropna(subset=["cdi"]))

df.set_index("Data")["cdi"].sort_index()

HTTPError: 404 Client Error: Not Found for url: https://api.bcb.gov.br/dados/serie/bcdata.sgs/12/dados?formato=json

In [5]:
def load_cdi_series(cache_csv: str = "Dados/cdi_cached.csv") -> pd.Series:
    """
    Retorna Series diária do CDI (decimal) indexada por datetime.

    1) Tenta ler o cache CSV (muito mais rápido).
    2) Se não existir ou estiver vazio, faz download da série 12 do SGS
       via requests e grava o cache.
    """
    # ───────── tenta cache ───────────────────────────────────────
    try:
        s = (pd.read_csv(cache_csv, parse_dates=["Data"])
                .set_index("Data")["cdi"]
                .astype(float)
                .sort_index())
        if not s.empty:
            return s
    except FileNotFoundError:
        pass                                          # segue para o download

    # ───────── download via API do BCB ───────────────────────────
    SGS_ID  = 12                      # CDI Over / Taxa DI
    dt_ini  = "2000-01-01"            # pode ajustar
    dt_fim  = dt.date.today().strftime("%Y-%m-%d")

    url = (f"https://api.bcb.gov.br/dados/serie/bcdata.sgs/{SGS_ID}/dados"
           f"?formato=json&dataInicial={dt_ini}&dataFinal={dt_fim}")

    resp = requests.get(url, timeout=15)
    resp.raise_for_status()           # dispara se 4xx/5xx

    df = (pd.DataFrame(resp.json())          # [{'data':'01/07/2024','valor':'0.0155'}, …]
            .rename(columns={"data": "Data", "valor": "cdi"})
            .assign(Data=lambda d: pd.to_datetime(d["Data"], dayfirst=True),
                    cdi =lambda d: pd.to_numeric(d["cdi"],
                                                 errors="coerce") / 100)   # 1,55% → 0.0155
            .dropna(subset=["cdi"]))

    serie = df.set_index("Data")["cdi"].sort_index()

    # guarda cache para os próximos 24 h
    serie.to_csv(cache_csv, index=True)

    return serie
df = load_cdi_series()
df

Data
2025-01-02    0.000455
2025-01-03    0.000455
2025-01-06    0.000455
2025-01-07    0.000455
2025-01-08    0.000455
                ...   
2025-07-07    0.000551
2025-07-08    0.000551
2025-07-09    0.000551
2025-07-10    0.000551
2025-07-11    0.000551
Name: cdi, Length: 131, dtype: float64

In [1]:
import pandas as pd

In [5]:
dados = pd.read_parquet("Dados/pl_fundos_teste.parquet")
dados

,Fundos/Carteiras Adm,2025-01-01,2025-01-02,2025-01-03,2025-01-04,2025-01-05,2025-01-06,2025-01-07,2025-01-08,2025-01-09,...,2025-10-28,2025-10-29,2025-10-30,2025-10-31,2025-11-01,2025-11-02,2025-11-03,2025-11-04,2025-11-05,Último Valor
0,AF DEB INCENTIVADAS,--,"R$ 11.703.717,60","R$ 11.725.635,32","R$ 11.725.635,32","R$ 11.725.635,32","R$ 11.772.948,06","R$ 11.748.277,03","R$ 11.771.658,07","R$ 11.776.481,27",...,"R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00"
1,AF INVEST GERAES PREV,--,"R$ 4.423.740,40","R$ 4.425.384,80","R$ 4.425.384,80","R$ 4.425.384,80","R$ 4.428.581,04","R$ 4.428.511,79","R$ 4.429.899,40","R$ 4.432.185,52",...,"R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00","R$ 0,00"
2,AF TÁTICO,--,"R$ 44.412.138,51","R$ 44.409.219,12","R$ 44.409.219,12","R$ 44.409.219,12","R$ 44.409.318,68","R$ 44.520.326,57","R$ 44.532.522,19","R$ 44.469.022,97",...,"R$ 51.674.787,98","R$ 51.721.433,21","R$ 51.741.916,26","R$ 51.810.880,37","R$ 51.741.916,26","R$ 51.741.916,26","R$ 51.863.950,20","R$ 51.863.950,20","R$ 51.863.950,20","R$ 51.863.950,20"
3,ALFA HORIZON FIA,--,"R$ 10.881.126,26","R$ 10.744.593,60","R$ 10.744.593,60","R$ 10.744.593,60","R$ 11.083.157,95","R$ 11.199.559,71","R$ 11.064.280,19","R$ 11.024.834,27",...,"R$ 11.101.713,90","R$ 11.075.110,12","R$ 11.142.559,96","R$ 11.378.697,76","R$ 11.378.697,76","R$ 11.378.697,76","R$ 11.374.029,09","R$ 11.534.319,40","R$ 11.534.319,40","R$ 11.534.319,40"
4,AYA NMK FIM,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
5,BBRASIL FIM CP RESP,--,"R$ 98.044.971,94","R$ 98.097.976,50","R$ 98.097.976,50","R$ 98.097.976,50","R$ 98.162.809,67","R$ 98.221.846,70","R$ 98.264.899,02","R$ 98.306.835,38",...,"R$ 110.578.038,02","R$ 110.638.091,15","R$ 110.698.048,87","R$ 110.758.368,87","R$ 110.758.368,87","R$ 110.758.368,87","R$ 110.818.574,13","R$ 110.818.574,13","R$ 110.818.574,13","R$ 110.818.574,13"
6,BH FIM,--,--,--,--,--,--,--,--,--,...,--,--,--,--,--,--,--,--,--,--
7,BH FIRF INFRA,--,"R$ 117.842.340,86","R$ 117.906.205,46","R$ 117.906.205,46","R$ 117.906.205,46","R$ 117.938.449,61","R$ 118.046.932,88","R$ 118.149.635,17","R$ 118.221.688,45",...,"R$ 213.567.672,90","R$ 213.655.636,08","R$ 213.587.707,82","R$ 213.734.887,31","R$ 213.587.707,82","R$ 213.587.707,82","R$ 213.696.301,26","R$ 213.696.301,26","R$ 213.696.301,26","R$ 213.696.301,26"
8,BMG SEG,--,"R$ 115.757.838,53","R$ 115.821.667,94","R$ 115.821.667,94","R$ 115.821.667,94","R$ 115.897.515,31","R$ 115.966.632,12","R$ 116.016.594,17","R$ 116.066.820,53",...,"R$ 128.928.847,45","R$ 128.993.974,87","R$ 129.060.966,44","R$ 129.134.742,01","R$ 129.134.742,01","R$ 129.134.742,01","R$ 129.202.504,46","R$ 129.272.211,16","R$ 129.272.211,16","R$ 129.272.211,16"
9,BORDEAUX FIM,--,"R$ 1.658.989,06","R$ 1.659.108,46","R$ 1.659.108,46","R$ 1.659.108,46","R$ 1.661.267,76","R$ 1.659.512,83","R$ 1.658.326,53","R$ 1.658.132,59",...,"R$ 0,08","R$ 0,08","R$ 0,08","R$ 0,08","R$ 0,08","R$ 0,08","R$ 0,08","R$ 0,08","R$ 0,08","R$ 0,08"
